In [7]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib 
import pandas as pd

Using matplotlib backend: MacOSX


In [8]:
x_data =np.linspace(0.0,10.0, 100000)

In [9]:
noise = np.random.randn(len(x_data))

In [10]:
noise

array([ 0.33666898,  0.69407209,  1.747946  , ..., -0.55337995,
        0.35778498,  0.26403084])

# Line 
y = mx+b
b =5

In [11]:
y_true = (0.5*x_data) +5 +noise

In [26]:
x_df = pd.DataFrame(data=x_data, columns=['X'])

In [27]:
y_df = pd.DataFrame(data=y_true, columns=['Y'])

In [28]:
print(y_df.head())

          Y
0  5.336669
1  5.694122
2  6.748046
3  5.346213
4  6.019579


In [29]:

my_data = pd.concat([x_df,y_df],axis=1)

In [30]:
my_data.head()


,X,Y
0,0.0000,5.336669
1,0.0001,5.694122
2,0.0002,6.748046
3,0.0003,5.346213
4,0.0004,6.019579


In [35]:
my_data.sample(n=250).plot(kind='scatter', x='X', y='Y')


In [36]:
'''
Let's create the batches'''

"\nLet's create the batches"

In [37]:
batch_size =8

In [38]:
m = tf.Variable(0.5)
b= tf.Variable(0.8)

In [39]:
# placeholder
xph = tf.placeholder(tf.float32,[batch_size])
yph =tf.placeholder(tf.float32, [batch_size])

In [40]:
y_model = m*xph +b

In [41]:
error = tf.reduce_sum(tf.square(yph-y_model))

In [42]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
train = optimizer.minimize(error)

In [43]:
init = tf.global_variables_initializer()

In [46]:
with tf.Session() as sess:
    sess.run(init)
    batches = 1000
    for i in range(batches):
        rand_ind = np.random.randint(len(x_data), size=batch_size)
        feed = {xph:x_data[rand_ind], yph:y_true[rand_ind]}
        sess.run(train, feed_dict=feed)
    model_m , model_b = sess.run([m,b])

model_m

In [47]:
model_m

0.47920802

In [48]:
model_b

4.899416

In [49]:
y_hat = x_data*model_m+model_b

In [50]:
my_data.sample(n=250).plot(kind='scatter',x='X', y='Y')
plt.plot(x_data,y_hat,'r')

In [51]:
''' For the final graph please check the .png file in the folder '''

' For the final graph please check the .png file in the folder '

In [52]:

''' Let's use the estimator API'''
feat_col = [tf.feature_column.numeric_column('x', shape = [1])]



In [53]:
estimator = tf.estimator.LinearRegressor(feature_columns=feat_col)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/y_/n4k0djsx7kg6tt7q600grqf80000gn/T/tmpv2kk7g__', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x182e1c3908>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [55]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_true,
                                                    test_size =0.2)



In [56]:

''' We need an input function which can act like a dict which can take feed fucntion and batch size at once'''
input_func = tf.estimator.inputs.numpy_input_fn({'x':x_train}, y_train,
                                          batch_size=8,
                                          num_epochs=None, shuffle=True)

In [57]:
train_input_func = tf.estimator.inputs.numpy_input_fn({'x':x_train}, y_train,
                                          batch_size=8,
                                          num_epochs=None, shuffle=False)
#shuffle false because we will use this train_input_function for evaluation against the test evaluation function 

In [58]:
test_input_func = tf.estimator.inputs.numpy_input_fn({'x':x_test}, y_test,
                                          batch_size=8,
                                          num_epochs=None, shuffle=False)

In [59]:
estimator.train(input_func, steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/y_/n4k0djsx7kg6tt7q600grqf80000gn/T/tmpv2kk7g__/model.ckpt.
INFO:tensorflow:loss = 607.8, step = 1
INFO:tensorflow:global_step/sec: 660.006
INFO:tensorflow:loss = 11.4829, step = 101 (0.155 sec)
INFO:tensorflow:global_step/sec: 556.286
INFO:tensorflow:loss = 28.6893, step = 201 (0.180 sec)
INFO:tensorflow:global_step/sec: 560.051
INFO:tensorflow:loss = 8.73281, step = 301 (0.177 sec)
INFO:tensorflow:global_step/sec: 447.882
INFO:tensorflow:loss = 6.29232, step = 401 (0.227 sec)
INFO:tensorflow:global_step/sec: 519.729
INFO:tensorflow:loss = 13.1831, step = 501 (0.188 sec)
INFO:tensorflow:global_step/sec: 510.319
INFO:tensorflow:loss = 27.9009, step = 601 (0.197 sec)
INFO:tensorflow:global_step/sec: 808.028
INFO:tensorflow:loss = 10.1916, step = 701 (0.123 sec)
INFO:tensorflow:global_step/sec: 640.361
INFO:tensorflow:loss = 12.1518, step = 801 (0.158 sec)
INFO:tensorflow:global_step/se

In [60]:
# Evaluation Matrics
train_metrics = estimator.evaluate(input_fn=train_input_func, steps=2000)

INFO:tensorflow:Starting evaluation at 2018-02-26-01:03:39
INFO:tensorflow:Restoring parameters from /var/folders/y_/n4k0djsx7kg6tt7q600grqf80000gn/T/tmpv2kk7g__/model.ckpt-1000
INFO:tensorflow:Evaluation [1/2000]
INFO:tensorflow:Evaluation [2/2000]
INFO:tensorflow:Evaluation [3/2000]
INFO:tensorflow:Evaluation [4/2000]
INFO:tensorflow:Evaluation [5/2000]
INFO:tensorflow:Evaluation [6/2000]
INFO:tensorflow:Evaluation [7/2000]
INFO:tensorflow:Evaluation [8/2000]
INFO:tensorflow:Evaluation [9/2000]
INFO:tensorflow:Evaluation [10/2000]
INFO:tensorflow:Evaluation [11/2000]
INFO:tensorflow:Evaluation [12/2000]
INFO:tensorflow:Evaluation [13/2000]
INFO:tensorflow:Evaluation [14/2000]
INFO:tensorflow:Evaluation [15/2000]
INFO:tensorflow:Evaluation [16/2000]
INFO:tensorflow:Evaluation [17/2000]
INFO:tensorflow:Evaluation [18/2000]
INFO:tensorflow:Evaluation [19/2000]
INFO:tensorflow:Evaluation [20/2000]
INFO:tensorflow:Evaluation [21/2000]
INFO:tensorflow:Evaluation [22/2000]
INFO:tensorflow:E

INFO:tensorflow:Evaluation [214/2000]
INFO:tensorflow:Evaluation [215/2000]
INFO:tensorflow:Evaluation [216/2000]
INFO:tensorflow:Evaluation [217/2000]
INFO:tensorflow:Evaluation [218/2000]
INFO:tensorflow:Evaluation [219/2000]
INFO:tensorflow:Evaluation [220/2000]
INFO:tensorflow:Evaluation [221/2000]
INFO:tensorflow:Evaluation [222/2000]
INFO:tensorflow:Evaluation [223/2000]
INFO:tensorflow:Evaluation [224/2000]
INFO:tensorflow:Evaluation [225/2000]
INFO:tensorflow:Evaluation [226/2000]
INFO:tensorflow:Evaluation [227/2000]
INFO:tensorflow:Evaluation [228/2000]
INFO:tensorflow:Evaluation [229/2000]
INFO:tensorflow:Evaluation [230/2000]
INFO:tensorflow:Evaluation [231/2000]
INFO:tensorflow:Evaluation [232/2000]
INFO:tensorflow:Evaluation [233/2000]
INFO:tensorflow:Evaluation [234/2000]
INFO:tensorflow:Evaluation [235/2000]
INFO:tensorflow:Evaluation [236/2000]
INFO:tensorflow:Evaluation [237/2000]
INFO:tensorflow:Evaluation [238/2000]
INFO:tensorflow:Evaluation [239/2000]
INFO:tensorf

INFO:tensorflow:Evaluation [429/2000]
INFO:tensorflow:Evaluation [430/2000]
INFO:tensorflow:Evaluation [431/2000]
INFO:tensorflow:Evaluation [432/2000]
INFO:tensorflow:Evaluation [433/2000]
INFO:tensorflow:Evaluation [434/2000]
INFO:tensorflow:Evaluation [435/2000]
INFO:tensorflow:Evaluation [436/2000]
INFO:tensorflow:Evaluation [437/2000]
INFO:tensorflow:Evaluation [438/2000]
INFO:tensorflow:Evaluation [439/2000]
INFO:tensorflow:Evaluation [440/2000]
INFO:tensorflow:Evaluation [441/2000]
INFO:tensorflow:Evaluation [442/2000]
INFO:tensorflow:Evaluation [443/2000]
INFO:tensorflow:Evaluation [444/2000]
INFO:tensorflow:Evaluation [445/2000]
INFO:tensorflow:Evaluation [446/2000]
INFO:tensorflow:Evaluation [447/2000]
INFO:tensorflow:Evaluation [448/2000]
INFO:tensorflow:Evaluation [449/2000]
INFO:tensorflow:Evaluation [450/2000]
INFO:tensorflow:Evaluation [451/2000]
INFO:tensorflow:Evaluation [452/2000]
INFO:tensorflow:Evaluation [453/2000]
INFO:tensorflow:Evaluation [454/2000]
INFO:tensorf

INFO:tensorflow:Evaluation [644/2000]
INFO:tensorflow:Evaluation [645/2000]
INFO:tensorflow:Evaluation [646/2000]
INFO:tensorflow:Evaluation [647/2000]
INFO:tensorflow:Evaluation [648/2000]
INFO:tensorflow:Evaluation [649/2000]
INFO:tensorflow:Evaluation [650/2000]
INFO:tensorflow:Evaluation [651/2000]
INFO:tensorflow:Evaluation [652/2000]
INFO:tensorflow:Evaluation [653/2000]
INFO:tensorflow:Evaluation [654/2000]
INFO:tensorflow:Evaluation [655/2000]
INFO:tensorflow:Evaluation [656/2000]
INFO:tensorflow:Evaluation [657/2000]
INFO:tensorflow:Evaluation [658/2000]
INFO:tensorflow:Evaluation [659/2000]
INFO:tensorflow:Evaluation [660/2000]
INFO:tensorflow:Evaluation [661/2000]
INFO:tensorflow:Evaluation [662/2000]
INFO:tensorflow:Evaluation [663/2000]
INFO:tensorflow:Evaluation [664/2000]
INFO:tensorflow:Evaluation [665/2000]
INFO:tensorflow:Evaluation [666/2000]
INFO:tensorflow:Evaluation [667/2000]
INFO:tensorflow:Evaluation [668/2000]
INFO:tensorflow:Evaluation [669/2000]
INFO:tensorf

INFO:tensorflow:Evaluation [860/2000]
INFO:tensorflow:Evaluation [861/2000]
INFO:tensorflow:Evaluation [862/2000]
INFO:tensorflow:Evaluation [863/2000]
INFO:tensorflow:Evaluation [864/2000]
INFO:tensorflow:Evaluation [865/2000]
INFO:tensorflow:Evaluation [866/2000]
INFO:tensorflow:Evaluation [867/2000]
INFO:tensorflow:Evaluation [868/2000]
INFO:tensorflow:Evaluation [869/2000]
INFO:tensorflow:Evaluation [870/2000]
INFO:tensorflow:Evaluation [871/2000]
INFO:tensorflow:Evaluation [872/2000]
INFO:tensorflow:Evaluation [873/2000]
INFO:tensorflow:Evaluation [874/2000]
INFO:tensorflow:Evaluation [875/2000]
INFO:tensorflow:Evaluation [876/2000]
INFO:tensorflow:Evaluation [877/2000]
INFO:tensorflow:Evaluation [878/2000]
INFO:tensorflow:Evaluation [879/2000]
INFO:tensorflow:Evaluation [880/2000]
INFO:tensorflow:Evaluation [881/2000]
INFO:tensorflow:Evaluation [882/2000]
INFO:tensorflow:Evaluation [883/2000]
INFO:tensorflow:Evaluation [884/2000]
INFO:tensorflow:Evaluation [885/2000]
INFO:tensorf

INFO:tensorflow:Evaluation [1074/2000]
INFO:tensorflow:Evaluation [1075/2000]
INFO:tensorflow:Evaluation [1076/2000]
INFO:tensorflow:Evaluation [1077/2000]
INFO:tensorflow:Evaluation [1078/2000]
INFO:tensorflow:Evaluation [1079/2000]
INFO:tensorflow:Evaluation [1080/2000]
INFO:tensorflow:Evaluation [1081/2000]
INFO:tensorflow:Evaluation [1082/2000]
INFO:tensorflow:Evaluation [1083/2000]
INFO:tensorflow:Evaluation [1084/2000]
INFO:tensorflow:Evaluation [1085/2000]
INFO:tensorflow:Evaluation [1086/2000]
INFO:tensorflow:Evaluation [1087/2000]
INFO:tensorflow:Evaluation [1088/2000]
INFO:tensorflow:Evaluation [1089/2000]
INFO:tensorflow:Evaluation [1090/2000]
INFO:tensorflow:Evaluation [1091/2000]
INFO:tensorflow:Evaluation [1092/2000]
INFO:tensorflow:Evaluation [1093/2000]
INFO:tensorflow:Evaluation [1094/2000]
INFO:tensorflow:Evaluation [1095/2000]
INFO:tensorflow:Evaluation [1096/2000]
INFO:tensorflow:Evaluation [1097/2000]
INFO:tensorflow:Evaluation [1098/2000]
INFO:tensorflow:Evaluatio

INFO:tensorflow:Evaluation [1284/2000]
INFO:tensorflow:Evaluation [1285/2000]
INFO:tensorflow:Evaluation [1286/2000]
INFO:tensorflow:Evaluation [1287/2000]
INFO:tensorflow:Evaluation [1288/2000]
INFO:tensorflow:Evaluation [1289/2000]
INFO:tensorflow:Evaluation [1290/2000]
INFO:tensorflow:Evaluation [1291/2000]
INFO:tensorflow:Evaluation [1292/2000]
INFO:tensorflow:Evaluation [1293/2000]
INFO:tensorflow:Evaluation [1294/2000]
INFO:tensorflow:Evaluation [1295/2000]
INFO:tensorflow:Evaluation [1296/2000]
INFO:tensorflow:Evaluation [1297/2000]
INFO:tensorflow:Evaluation [1298/2000]
INFO:tensorflow:Evaluation [1299/2000]
INFO:tensorflow:Evaluation [1300/2000]
INFO:tensorflow:Evaluation [1301/2000]
INFO:tensorflow:Evaluation [1302/2000]
INFO:tensorflow:Evaluation [1303/2000]
INFO:tensorflow:Evaluation [1304/2000]
INFO:tensorflow:Evaluation [1305/2000]
INFO:tensorflow:Evaluation [1306/2000]
INFO:tensorflow:Evaluation [1307/2000]
INFO:tensorflow:Evaluation [1308/2000]
INFO:tensorflow:Evaluatio

INFO:tensorflow:Evaluation [1494/2000]
INFO:tensorflow:Evaluation [1495/2000]
INFO:tensorflow:Evaluation [1496/2000]
INFO:tensorflow:Evaluation [1497/2000]
INFO:tensorflow:Evaluation [1498/2000]
INFO:tensorflow:Evaluation [1499/2000]
INFO:tensorflow:Evaluation [1500/2000]
INFO:tensorflow:Evaluation [1501/2000]
INFO:tensorflow:Evaluation [1502/2000]
INFO:tensorflow:Evaluation [1503/2000]
INFO:tensorflow:Evaluation [1504/2000]
INFO:tensorflow:Evaluation [1505/2000]
INFO:tensorflow:Evaluation [1506/2000]
INFO:tensorflow:Evaluation [1507/2000]
INFO:tensorflow:Evaluation [1508/2000]
INFO:tensorflow:Evaluation [1509/2000]
INFO:tensorflow:Evaluation [1510/2000]
INFO:tensorflow:Evaluation [1511/2000]
INFO:tensorflow:Evaluation [1512/2000]
INFO:tensorflow:Evaluation [1513/2000]
INFO:tensorflow:Evaluation [1514/2000]
INFO:tensorflow:Evaluation [1515/2000]
INFO:tensorflow:Evaluation [1516/2000]
INFO:tensorflow:Evaluation [1517/2000]
INFO:tensorflow:Evaluation [1518/2000]
INFO:tensorflow:Evaluatio

INFO:tensorflow:Evaluation [1705/2000]
INFO:tensorflow:Evaluation [1706/2000]
INFO:tensorflow:Evaluation [1707/2000]
INFO:tensorflow:Evaluation [1708/2000]
INFO:tensorflow:Evaluation [1709/2000]
INFO:tensorflow:Evaluation [1710/2000]
INFO:tensorflow:Evaluation [1711/2000]
INFO:tensorflow:Evaluation [1712/2000]
INFO:tensorflow:Evaluation [1713/2000]
INFO:tensorflow:Evaluation [1714/2000]
INFO:tensorflow:Evaluation [1715/2000]
INFO:tensorflow:Evaluation [1716/2000]
INFO:tensorflow:Evaluation [1717/2000]
INFO:tensorflow:Evaluation [1718/2000]
INFO:tensorflow:Evaluation [1719/2000]
INFO:tensorflow:Evaluation [1720/2000]
INFO:tensorflow:Evaluation [1721/2000]
INFO:tensorflow:Evaluation [1722/2000]
INFO:tensorflow:Evaluation [1723/2000]
INFO:tensorflow:Evaluation [1724/2000]
INFO:tensorflow:Evaluation [1725/2000]
INFO:tensorflow:Evaluation [1726/2000]
INFO:tensorflow:Evaluation [1727/2000]
INFO:tensorflow:Evaluation [1728/2000]
INFO:tensorflow:Evaluation [1729/2000]
INFO:tensorflow:Evaluatio

INFO:tensorflow:Evaluation [1916/2000]
INFO:tensorflow:Evaluation [1917/2000]
INFO:tensorflow:Evaluation [1918/2000]
INFO:tensorflow:Evaluation [1919/2000]
INFO:tensorflow:Evaluation [1920/2000]
INFO:tensorflow:Evaluation [1921/2000]
INFO:tensorflow:Evaluation [1922/2000]
INFO:tensorflow:Evaluation [1923/2000]
INFO:tensorflow:Evaluation [1924/2000]
INFO:tensorflow:Evaluation [1925/2000]
INFO:tensorflow:Evaluation [1926/2000]
INFO:tensorflow:Evaluation [1927/2000]
INFO:tensorflow:Evaluation [1928/2000]
INFO:tensorflow:Evaluation [1929/2000]
INFO:tensorflow:Evaluation [1930/2000]
INFO:tensorflow:Evaluation [1931/2000]
INFO:tensorflow:Evaluation [1932/2000]
INFO:tensorflow:Evaluation [1933/2000]
INFO:tensorflow:Evaluation [1934/2000]
INFO:tensorflow:Evaluation [1935/2000]
INFO:tensorflow:Evaluation [1936/2000]
INFO:tensorflow:Evaluation [1937/2000]
INFO:tensorflow:Evaluation [1938/2000]
INFO:tensorflow:Evaluation [1939/2000]
INFO:tensorflow:Evaluation [1940/2000]
INFO:tensorflow:Evaluatio

In [61]:
test_matrics=estimator.evaluate(input_fn=test_input_func, steps=2000)

INFO:tensorflow:Starting evaluation at 2018-02-26-01:03:53
INFO:tensorflow:Restoring parameters from /var/folders/y_/n4k0djsx7kg6tt7q600grqf80000gn/T/tmpv2kk7g__/model.ckpt-1000
INFO:tensorflow:Evaluation [1/2000]
INFO:tensorflow:Evaluation [2/2000]
INFO:tensorflow:Evaluation [3/2000]
INFO:tensorflow:Evaluation [4/2000]
INFO:tensorflow:Evaluation [5/2000]
INFO:tensorflow:Evaluation [6/2000]
INFO:tensorflow:Evaluation [7/2000]
INFO:tensorflow:Evaluation [8/2000]
INFO:tensorflow:Evaluation [9/2000]
INFO:tensorflow:Evaluation [10/2000]
INFO:tensorflow:Evaluation [11/2000]
INFO:tensorflow:Evaluation [12/2000]
INFO:tensorflow:Evaluation [13/2000]
INFO:tensorflow:Evaluation [14/2000]
INFO:tensorflow:Evaluation [15/2000]
INFO:tensorflow:Evaluation [16/2000]
INFO:tensorflow:Evaluation [17/2000]
INFO:tensorflow:Evaluation [18/2000]
INFO:tensorflow:Evaluation [19/2000]
INFO:tensorflow:Evaluation [20/2000]
INFO:tensorflow:Evaluation [21/2000]
INFO:tensorflow:Evaluation [22/2000]
INFO:tensorflow:E

INFO:tensorflow:Evaluation [215/2000]
INFO:tensorflow:Evaluation [216/2000]
INFO:tensorflow:Evaluation [217/2000]
INFO:tensorflow:Evaluation [218/2000]
INFO:tensorflow:Evaluation [219/2000]
INFO:tensorflow:Evaluation [220/2000]
INFO:tensorflow:Evaluation [221/2000]
INFO:tensorflow:Evaluation [222/2000]
INFO:tensorflow:Evaluation [223/2000]
INFO:tensorflow:Evaluation [224/2000]
INFO:tensorflow:Evaluation [225/2000]
INFO:tensorflow:Evaluation [226/2000]
INFO:tensorflow:Evaluation [227/2000]
INFO:tensorflow:Evaluation [228/2000]
INFO:tensorflow:Evaluation [229/2000]
INFO:tensorflow:Evaluation [230/2000]
INFO:tensorflow:Evaluation [231/2000]
INFO:tensorflow:Evaluation [232/2000]
INFO:tensorflow:Evaluation [233/2000]
INFO:tensorflow:Evaluation [234/2000]
INFO:tensorflow:Evaluation [235/2000]
INFO:tensorflow:Evaluation [236/2000]
INFO:tensorflow:Evaluation [237/2000]
INFO:tensorflow:Evaluation [238/2000]
INFO:tensorflow:Evaluation [239/2000]
INFO:tensorflow:Evaluation [240/2000]
INFO:tensorf

INFO:tensorflow:Evaluation [431/2000]
INFO:tensorflow:Evaluation [432/2000]
INFO:tensorflow:Evaluation [433/2000]
INFO:tensorflow:Evaluation [434/2000]
INFO:tensorflow:Evaluation [435/2000]
INFO:tensorflow:Evaluation [436/2000]
INFO:tensorflow:Evaluation [437/2000]
INFO:tensorflow:Evaluation [438/2000]
INFO:tensorflow:Evaluation [439/2000]
INFO:tensorflow:Evaluation [440/2000]
INFO:tensorflow:Evaluation [441/2000]
INFO:tensorflow:Evaluation [442/2000]
INFO:tensorflow:Evaluation [443/2000]
INFO:tensorflow:Evaluation [444/2000]
INFO:tensorflow:Evaluation [445/2000]
INFO:tensorflow:Evaluation [446/2000]
INFO:tensorflow:Evaluation [447/2000]
INFO:tensorflow:Evaluation [448/2000]
INFO:tensorflow:Evaluation [449/2000]
INFO:tensorflow:Evaluation [450/2000]
INFO:tensorflow:Evaluation [451/2000]
INFO:tensorflow:Evaluation [452/2000]
INFO:tensorflow:Evaluation [453/2000]
INFO:tensorflow:Evaluation [454/2000]
INFO:tensorflow:Evaluation [455/2000]
INFO:tensorflow:Evaluation [456/2000]
INFO:tensorf

INFO:tensorflow:Evaluation [646/2000]
INFO:tensorflow:Evaluation [647/2000]
INFO:tensorflow:Evaluation [648/2000]
INFO:tensorflow:Evaluation [649/2000]
INFO:tensorflow:Evaluation [650/2000]
INFO:tensorflow:Evaluation [651/2000]
INFO:tensorflow:Evaluation [652/2000]
INFO:tensorflow:Evaluation [653/2000]
INFO:tensorflow:Evaluation [654/2000]
INFO:tensorflow:Evaluation [655/2000]
INFO:tensorflow:Evaluation [656/2000]
INFO:tensorflow:Evaluation [657/2000]
INFO:tensorflow:Evaluation [658/2000]
INFO:tensorflow:Evaluation [659/2000]
INFO:tensorflow:Evaluation [660/2000]
INFO:tensorflow:Evaluation [661/2000]
INFO:tensorflow:Evaluation [662/2000]
INFO:tensorflow:Evaluation [663/2000]
INFO:tensorflow:Evaluation [664/2000]
INFO:tensorflow:Evaluation [665/2000]
INFO:tensorflow:Evaluation [666/2000]
INFO:tensorflow:Evaluation [667/2000]
INFO:tensorflow:Evaluation [668/2000]
INFO:tensorflow:Evaluation [669/2000]
INFO:tensorflow:Evaluation [670/2000]
INFO:tensorflow:Evaluation [671/2000]
INFO:tensorf

INFO:tensorflow:Evaluation [862/2000]
INFO:tensorflow:Evaluation [863/2000]
INFO:tensorflow:Evaluation [864/2000]
INFO:tensorflow:Evaluation [865/2000]
INFO:tensorflow:Evaluation [866/2000]
INFO:tensorflow:Evaluation [867/2000]
INFO:tensorflow:Evaluation [868/2000]
INFO:tensorflow:Evaluation [869/2000]
INFO:tensorflow:Evaluation [870/2000]
INFO:tensorflow:Evaluation [871/2000]
INFO:tensorflow:Evaluation [872/2000]
INFO:tensorflow:Evaluation [873/2000]
INFO:tensorflow:Evaluation [874/2000]
INFO:tensorflow:Evaluation [875/2000]
INFO:tensorflow:Evaluation [876/2000]
INFO:tensorflow:Evaluation [877/2000]
INFO:tensorflow:Evaluation [878/2000]
INFO:tensorflow:Evaluation [879/2000]
INFO:tensorflow:Evaluation [880/2000]
INFO:tensorflow:Evaluation [881/2000]
INFO:tensorflow:Evaluation [882/2000]
INFO:tensorflow:Evaluation [883/2000]
INFO:tensorflow:Evaluation [884/2000]
INFO:tensorflow:Evaluation [885/2000]
INFO:tensorflow:Evaluation [886/2000]
INFO:tensorflow:Evaluation [887/2000]
INFO:tensorf

INFO:tensorflow:Evaluation [1075/2000]
INFO:tensorflow:Evaluation [1076/2000]
INFO:tensorflow:Evaluation [1077/2000]
INFO:tensorflow:Evaluation [1078/2000]
INFO:tensorflow:Evaluation [1079/2000]
INFO:tensorflow:Evaluation [1080/2000]
INFO:tensorflow:Evaluation [1081/2000]
INFO:tensorflow:Evaluation [1082/2000]
INFO:tensorflow:Evaluation [1083/2000]
INFO:tensorflow:Evaluation [1084/2000]
INFO:tensorflow:Evaluation [1085/2000]
INFO:tensorflow:Evaluation [1086/2000]
INFO:tensorflow:Evaluation [1087/2000]
INFO:tensorflow:Evaluation [1088/2000]
INFO:tensorflow:Evaluation [1089/2000]
INFO:tensorflow:Evaluation [1090/2000]
INFO:tensorflow:Evaluation [1091/2000]
INFO:tensorflow:Evaluation [1092/2000]
INFO:tensorflow:Evaluation [1093/2000]
INFO:tensorflow:Evaluation [1094/2000]
INFO:tensorflow:Evaluation [1095/2000]
INFO:tensorflow:Evaluation [1096/2000]
INFO:tensorflow:Evaluation [1097/2000]
INFO:tensorflow:Evaluation [1098/2000]
INFO:tensorflow:Evaluation [1099/2000]
INFO:tensorflow:Evaluatio

INFO:tensorflow:Evaluation [1286/2000]
INFO:tensorflow:Evaluation [1287/2000]
INFO:tensorflow:Evaluation [1288/2000]
INFO:tensorflow:Evaluation [1289/2000]
INFO:tensorflow:Evaluation [1290/2000]
INFO:tensorflow:Evaluation [1291/2000]
INFO:tensorflow:Evaluation [1292/2000]
INFO:tensorflow:Evaluation [1293/2000]
INFO:tensorflow:Evaluation [1294/2000]
INFO:tensorflow:Evaluation [1295/2000]
INFO:tensorflow:Evaluation [1296/2000]
INFO:tensorflow:Evaluation [1297/2000]
INFO:tensorflow:Evaluation [1298/2000]
INFO:tensorflow:Evaluation [1299/2000]
INFO:tensorflow:Evaluation [1300/2000]
INFO:tensorflow:Evaluation [1301/2000]
INFO:tensorflow:Evaluation [1302/2000]
INFO:tensorflow:Evaluation [1303/2000]
INFO:tensorflow:Evaluation [1304/2000]
INFO:tensorflow:Evaluation [1305/2000]
INFO:tensorflow:Evaluation [1306/2000]
INFO:tensorflow:Evaluation [1307/2000]
INFO:tensorflow:Evaluation [1308/2000]
INFO:tensorflow:Evaluation [1309/2000]
INFO:tensorflow:Evaluation [1310/2000]
INFO:tensorflow:Evaluatio

INFO:tensorflow:Evaluation [1496/2000]
INFO:tensorflow:Evaluation [1497/2000]
INFO:tensorflow:Evaluation [1498/2000]
INFO:tensorflow:Evaluation [1499/2000]
INFO:tensorflow:Evaluation [1500/2000]
INFO:tensorflow:Evaluation [1501/2000]
INFO:tensorflow:Evaluation [1502/2000]
INFO:tensorflow:Evaluation [1503/2000]
INFO:tensorflow:Evaluation [1504/2000]
INFO:tensorflow:Evaluation [1505/2000]
INFO:tensorflow:Evaluation [1506/2000]
INFO:tensorflow:Evaluation [1507/2000]
INFO:tensorflow:Evaluation [1508/2000]
INFO:tensorflow:Evaluation [1509/2000]
INFO:tensorflow:Evaluation [1510/2000]
INFO:tensorflow:Evaluation [1511/2000]
INFO:tensorflow:Evaluation [1512/2000]
INFO:tensorflow:Evaluation [1513/2000]
INFO:tensorflow:Evaluation [1514/2000]
INFO:tensorflow:Evaluation [1515/2000]
INFO:tensorflow:Evaluation [1516/2000]
INFO:tensorflow:Evaluation [1517/2000]
INFO:tensorflow:Evaluation [1518/2000]
INFO:tensorflow:Evaluation [1519/2000]
INFO:tensorflow:Evaluation [1520/2000]
INFO:tensorflow:Evaluatio

INFO:tensorflow:Evaluation [1707/2000]
INFO:tensorflow:Evaluation [1708/2000]
INFO:tensorflow:Evaluation [1709/2000]
INFO:tensorflow:Evaluation [1710/2000]
INFO:tensorflow:Evaluation [1711/2000]
INFO:tensorflow:Evaluation [1712/2000]
INFO:tensorflow:Evaluation [1713/2000]
INFO:tensorflow:Evaluation [1714/2000]
INFO:tensorflow:Evaluation [1715/2000]
INFO:tensorflow:Evaluation [1716/2000]
INFO:tensorflow:Evaluation [1717/2000]
INFO:tensorflow:Evaluation [1718/2000]
INFO:tensorflow:Evaluation [1719/2000]
INFO:tensorflow:Evaluation [1720/2000]
INFO:tensorflow:Evaluation [1721/2000]
INFO:tensorflow:Evaluation [1722/2000]
INFO:tensorflow:Evaluation [1723/2000]
INFO:tensorflow:Evaluation [1724/2000]
INFO:tensorflow:Evaluation [1725/2000]
INFO:tensorflow:Evaluation [1726/2000]
INFO:tensorflow:Evaluation [1727/2000]
INFO:tensorflow:Evaluation [1728/2000]
INFO:tensorflow:Evaluation [1729/2000]
INFO:tensorflow:Evaluation [1730/2000]
INFO:tensorflow:Evaluation [1731/2000]
INFO:tensorflow:Evaluatio

INFO:tensorflow:Evaluation [1917/2000]
INFO:tensorflow:Evaluation [1918/2000]
INFO:tensorflow:Evaluation [1919/2000]
INFO:tensorflow:Evaluation [1920/2000]
INFO:tensorflow:Evaluation [1921/2000]
INFO:tensorflow:Evaluation [1922/2000]
INFO:tensorflow:Evaluation [1923/2000]
INFO:tensorflow:Evaluation [1924/2000]
INFO:tensorflow:Evaluation [1925/2000]
INFO:tensorflow:Evaluation [1926/2000]
INFO:tensorflow:Evaluation [1927/2000]
INFO:tensorflow:Evaluation [1928/2000]
INFO:tensorflow:Evaluation [1929/2000]
INFO:tensorflow:Evaluation [1930/2000]
INFO:tensorflow:Evaluation [1931/2000]
INFO:tensorflow:Evaluation [1932/2000]
INFO:tensorflow:Evaluation [1933/2000]
INFO:tensorflow:Evaluation [1934/2000]
INFO:tensorflow:Evaluation [1935/2000]
INFO:tensorflow:Evaluation [1936/2000]
INFO:tensorflow:Evaluation [1937/2000]
INFO:tensorflow:Evaluation [1938/2000]
INFO:tensorflow:Evaluation [1939/2000]
INFO:tensorflow:Evaluation [1940/2000]
INFO:tensorflow:Evaluation [1941/2000]
INFO:tensorflow:Evaluatio

In [62]:
print('Training Data Metrics')
print(train_metrics)

Training Data Metrics
{'average_loss': 1.117039, 'loss': 8.9363117, 'global_step': 1000}


In [63]:
print('Testing Data Metrics')
print(test_matrics)

Testing Data Metrics
{'average_loss': 1.0923874, 'loss': 8.7390995, 'global_step': 1000}


# General note on evaluation of data
To check the overfitting in matrics evaluation. The main indicator is when the training loss > testing loss. if (train loss > testing loss): overfitting :)
The training and testing loss should be close as much as possible.

In [64]:
# how to predict the new values ?
new_data = np.linspace(0,10,10)



In [65]:
input_fn_predict = tf.estimator.inputs.numpy_input_fn({'x':new_data},
                                                     shuffle=False)

In [66]:
list(estimator.predict(input_fn=input_fn_predict))

INFO:tensorflow:Restoring parameters from /var/folders/y_/n4k0djsx7kg6tt7q600grqf80000gn/T/tmpv2kk7g__/model.ckpt-1000


[{'predictions': array([ 4.38921118], dtype=float32)},
 {'predictions': array([ 5.06129646], dtype=float32)},
 {'predictions': array([ 5.73338175], dtype=float32)},
 {'predictions': array([ 6.40546703], dtype=float32)},
 {'predictions': array([ 7.0775528], dtype=float32)},
 {'predictions': array([ 7.7496376], dtype=float32)},
 {'predictions': array([ 8.42172241], dtype=float32)},
 {'predictions': array([ 9.09380817], dtype=float32)},
 {'predictions': array([ 9.76589394], dtype=float32)},
 {'predictions': array([ 10.43797874], dtype=float32)}]

In [67]:
prediction =[]
for pred in estimator.predict(input_fn=input_fn_predict):
    prediction.append(pred['predictions'])

INFO:tensorflow:Restoring parameters from /var/folders/y_/n4k0djsx7kg6tt7q600grqf80000gn/T/tmpv2kk7g__/model.ckpt-1000


In [68]:
prediction

[array([ 4.38921118], dtype=float32),
 array([ 5.06129646], dtype=float32),
 array([ 5.73338175], dtype=float32),
 array([ 6.40546703], dtype=float32),
 array([ 7.0775528], dtype=float32),
 array([ 7.7496376], dtype=float32),
 array([ 8.42172241], dtype=float32),
 array([ 9.09380817], dtype=float32),
 array([ 9.76589394], dtype=float32),
 array([ 10.43797874], dtype=float32)]

In [69]:
my_data.sample(n=250).plot(kind='scatter', x='X', y='Y')